<h2>Building a LangChain Agent with Tools<h2>

pip install wikipedia serpapi


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.agents.load_tools import Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

In [ ]:
# search = DuckDuckGoSearchRun()

# def multiply_numbers(a,b):
#     return a*b

# tools = [Tool(
#                 name = "testfun",
#                 func=search.run,
#                 description="internate search"
#             ),
#             Tool(
#                 name="Multiplier",
#                 func=lambda x: multiply_numbers(*x.split()),
#                 description="Multiplies two numbers. Input format: '3 4'"

#             )
#          ]

In [12]:
search = DuckDuckGoSearchRun()

def multiply_numbers(a: str, b: str) -> str:
    return str(float(a) * float(b))

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for when you need to look something up on the internet"
    ),
    Tool(
        name="Multiplier",
        func=lambda x: multiply_numbers(*x.split()),
        description="Multiplies two numbers. Input format: '3 4'"
    )
]


In [14]:
from langchain.chat_models import ChatOpenAI
from os import getenv

llm = ChatOpenAI(
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    model_name="google/gemma-3-27b-it:free",
)

response = llm.invoke("where is india in world")

In [15]:
response

AIMessage(content="India is located in **South Asia**. Here's a more detailed breakdown of its location:\n\n* **Continent:** Asia\n* **Region:** South Asia\n* **Coordinates:** Roughly 20.5937° N latitude and 78.9629° E longitude.  (This is a general center point, as India is a large country!)\n* **Borders:**\n    * **North:** China, Nepal, Bhutan\n    * **West:** Pakistan, Arabian Sea\n    * **South:** Indian Ocean\n    * **East:** Bangladesh, Myanmar, Bay of Bengal\n* **Relative to other major countries:**\n    *  It's northeast of Africa, northwest of Australia, and east of the Middle East.\n    *  It's a significant distance from Europe and the Americas.\n\n**Here's a helpful way to visualize it:**\n\n[https://www.worldometers.info/world-countries/india/](https://www.worldometers.info/world-countries/india/)  (This link has a map!)\n\nYou can also easily find it on any world map or using online map services like Google Maps.\n\n\n\n")

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


response = agent.run("what is capital of india, and what is 2 times 3")
response




> Entering new AgentExecutor chain...

Thought: The question has two parts. First, I need to find the capital of India. Second, I need to multiply 2 and 3. I will use DuckDuckGo Search for the first part and the Multiplier tool for the second part.
Action: DuckDuckGo Search
Action Input: capital of india
Observation: New Delhi is the capital of India since 1931, when it replaced Calcutta (Kolkata). It is a modern city with broad avenues, green spaces, and government buildings, located south of Old Delhi. Learn about India, the world's most populous country and a constitutional republic with a rich cultural and religious heritage. Find out its capital, New Delhi, and other facts about its geography, economy, and history. Learn why New Delhi is the capital of India since 1911, and what was the original name and location of the capital before that. Find out more facts and trivia about India's history, geography, and culture in this article. 29 States of India and Their Capitals and Lang

'The capital of India is New Delhi, and 2 times 3 is 6.'

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from os import getenv

# LLM
llm = ChatOpenAI(
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    model_name="google/gemma-3-27b-it:free"
)

# LLMChain: generate a slogan
prompt = PromptTemplate(
    input_variables=["product"],
    template="Write a creative slogan for a product called {product}."
)

slogan_chain = LLMChain(llm=llm, prompt=prompt)

# Wrap LLMChain as a Tool
slogan_tool = Tool(
    name="SloganGenerator",
    func=slogan_chain.run,
    description="Generates a marketing slogan. Input: product name."
)

# Agent with the LLMChain tool
agent = initialize_agent(
    tools=[slogan_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
agent.run("Create a slogan for a solar-powered backpack.")
